<a href="https://colab.research.google.com/github/AhmedFarrukh/DeepLearning-EdgeComputing/blob/main/notebooks/MeasuringInferenceTimes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, the inference times and memory footprint of the original and quantized versions of 7 popular Convolutional Neural Networks are measured and compared.

The CNN models are: MobileNet, InceptionV3, Resnet50, ResNet101, ResNet152, VGG16, VGG19.

The quantized models were created by applying [Post-training Dynamic Range Quantization](https://www.tensorflow.org/lite/performance/post_training_quantization).

Both the original models, and their quantized versions, are of tflite format, and were uploaded to [Google Drive](https://drive.google.com/drive/folders/1OcJ9ceYg6ZWFJ4QMR0zznsw0KVeHPa4h?usp=drive_link).

The benchmarking of models is achieved by using the official [TFlite benchmark](https://www.tensorflow.org/lite/performance/measurement) which measures the following metrics:
*   Initialization time
*   Inference time of warmup state
*   Inference time of steady state
*   Memory usage during initialization time
*   Overall memory usage

The benchmark generates a series of random inputs, runs the models and aggregates the results to report the aforementioned metrics.

In [ ]:
modelNames = ["MobileNet", "InceptionV3", "ResNet50", "ResNet101", "ResNet152", "VGG16", "VGG19"]

We can download the models from the Google Drive using gdown. If you want to download your own set of models, you can modify the google drive link below. In this case, we download the models to the /root/tflite_models directory.

In [ ]:
!pip install gdown

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1OcJ9ceYg6ZWFJ4QMR0zznsw0KVeHPa4h -O /root/tflite_models

You can verify that the models were correctly loaded by listing the files in the /root/tflite_models directory. Note that there should be two tflite files for each model: an original and a quantized version. The size of the quantized models should be significantly smaller than the size of their corresponding original model.

In [ ]:
!ls -l /root/tflite_models

Next, we download the TFlite benchmark which we will use to measure inference times and memory footprint. More details about the benchmark can be found on the [tensorflow website](https://www.tensorflow.org/lite/performance/measurement). Note that the benchmark is specific to the architecture type (such as x86 or ARM), and the appropriate benchmark binary must be downloaded. Below, the benchmark is loaded for an x86-64 type architecture.

The benchmark is downloaded to the /root/benchmark folder, and its permissions are then updated to allow it to be executed.

In [ ]:
!mkdir /root/benchmark
!wget https://storage.googleapis.com/tensorflow-nightly-public/prod/tensorflow/release/lite/tools/nightly/latest/linux_x86-64_benchmark_model -P /root/benchmark
!chmod +x /root/benchmark/linux_x86-64_benchmark_model

Let's run the benchmark on the MobileNet_quant model and note the output.

In [ ]:
!/root/benchmark/linux_x86-64_benchmark_model \
      --graph=/root/tflite_models/MobileNet_quant.tflite \
      --num_threads=1

Since the result of the benchmark is reported as text on the console, we can define a parsing function to extract the data. The parsing function takes the output of the benchmark as an input and adds the results to a dictionary of metrics.

In [ ]:
import re

def parse_benchmark_output(output, results):
    """
    Parse benchmark output to extract model initialization times, inference timings, and memory footprint.
    """

    # Regular expressions to match the required information
    init_time_patterns = [
        re.compile(r'INFO: Initialized session in (\d+.\d+)ms.'),
        re.compile(r'INFO: Initialized session in (\d+)ms.')
    ]
    inference_patterns = [
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): (\d+.\d+), Inference \(avg\): (\d+.\d+)'),
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): (\d+), Inference \(avg\): (\d+)'),
        re.compile(r'INFO: Inference timings in us: Init: (\d+), First inference: (\d+), Warmup \(avg\): ([\d.e+]+), Inference \(avg\): (\d+)')
    ]
    memory_pattern = re.compile(r'INFO: Memory footprint delta from the start of the tool \(MB\): init=(\d+.\d+) overall=(\d+.\d+)')

    for line in output.split('\n'):
        # Match the initialization time
        for pattern in init_time_patterns:
            init_match = pattern.search(line)
            if init_match:
                results['Init Time (ms)'].append(float(init_match.group(1)))
                break

        # Match the inference timings
        for pattern in inference_patterns:
            inference_match = pattern.search(line)
            if inference_match:
                results["Init Inference (ms)"].append(int(inference_match.group(1))/1000)
                results["First Inference (ms)"].append(int(inference_match.group(2))/1000)
                results["Warmup Inference (ms)"].append(float(inference_match.group(3))/1000)
                results["Avg Inference (ms)"].append(float(inference_match.group(4))/1000)
                break

        # Match the memory footprint
        memory_match = memory_pattern.search(line)
        if memory_match:
            results['Memory Init (MB)'].append(float(memory_match.group(1)))
            results['Memory Overall (MB)'].append(float(memory_match.group(2)))



Next, we can define a Pandas Dataframe to store our results. Since we will be repeatedly running the benchmark to estimate the standard deviation of results as well, for each metric, we will define two columns - one for the mean and the other for the standard deviation.

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

# Define model types (rows)
rows = []
for model in modelNames:
  rows.append(model)
  rows.append(model + "_quant")

# Define features (columns)
cols = []
features = ["Init Time (ms)", "Init Inference (ms)", "First Inference (ms)", "Warmup Inference (ms)", "Avg Inference (ms)", "Memory Init (MB)", "Memory Overall (MB)"]
for feature in features:
  cols.append(feature)
  cols.append(feature + "_sd")

# Create an empty DataFrame
finalResult = pd.DataFrame(index=rows, columns=cols)

Finally, run the benchmark repeatedly and average the results. For each model, we repeatedly run the benchmark, and parse the output from the benchmark. After `n` trials, the mean and standard deviation of the metrics is added to the `finalResult` dataframe defined in the last step.

In [ ]:
import subprocess
from collections import defaultdict
from statistics import mean
from statistics import stdev

n = 10 #the number of times the benchmark is called for each model

for modelName in rows:
  print(modelName)
  modelResults = defaultdict(list)
  for i in range(n):
    outputOriginal = subprocess.check_output("/root/benchmark/linux_x86-64_benchmark_model \
      --graph=/root/tflite_models/" + modelName +".tflite"+" \
      --num_threads=1", shell=True)
    outputOriginal = outputOriginal.decode('utf-8')
    output = parse_benchmark_output(outputOriginal, modelResults)

  for feature in features:
    finalResult.loc[modelName, feature] = mean(modelResults[feature])
    finalResult.loc[modelName, feature + "_sd"] = stdev(modelResults[feature])

Let's have a look at the results.

In [ ]:
print(finalResult)

Finally, we can generate plots of the results.

In [ ]:
!pip install matplotlib
!pip install numpy

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
for feature in features:
    means_orig = finalResult.loc[modelNames, feature].values
    errors_orig = finalResult.loc[modelNames, feature + "_sd"].values
    means_quant = finalResult.loc[[model + "_quant" for model in modelNames], feature].values
    errors_quant = finalResult.loc[[model + "_quant" for model in modelNames], feature + "_sd"].values


    n_groups = len(modelNames)
    index = np.arange(n_groups)

    fig, ax = plt.subplots()
    bar_width = 0.35
    opacity = 0.8

    rects1 = plt.bar(index, means_orig, bar_width,
                     alpha=opacity,
                     yerr=errors_orig,
                     label='Original')

    rects2 = plt.bar(index + bar_width, means_quant, bar_width,
                     alpha=opacity,
                     yerr=errors_quant,
                     label='Quantized')

    plt.xlabel('Model')
    plt.ylabel(feature)
    plt.title(f'Bar Chart for {feature}')
    plt.xticks(index + bar_width / 2, modelNames, rotation=45)
    plt.legend()

    plt.tight_layout()

    # Save the plot as an image
    plt.savefig("/root/plots" + feature + "_bar_chart.png")

    # Show the plot
    plt.show()